# Spårbredden hos en CD-skiva

![Spårbredden hos en CD-skiva](cdskiva.png)

Vilhelm Prytz  

Medlaboranter: Pontus Liedgren  
Handledare: Sanna Bodemyr  
Tullinge gymnasium  
2021-03-22

## Syfte

Syftet med denna undersökning är att bestämma spårbredden på en CD-skiva.

## Frågeställningar

Vilken spårbredd har CD-skivan?

## Reflektion/diskussion

För att bestämma spårbredden hos en CD-skiva går det att använda en laserpekare vars våglängd är känd (att ta reda på våglängden hos en laserpekare med okänd våglängd går att göra i en separat laboration), en CD-skiva, en linjal och stativmaterial.

Med hjälp av dessa material går det att utnyttja CD-skivan som ett gitter. Genom att montera CD-skivan i ett stativmaterial kan laserns ställas så att dess stråle träffar CD-skivan och reflekteras mot en plan yta. På ytan ska då flera laserpunkter synas eftersom gittret (CD-skivan) reflekterar och släpper igenom enligt ett visst mönster.

$d \cdot \sin{\alpha} = n \cdot \lambda$

I tabellen nedan visas data som antecknades under laborationen. I första kolumnen visas våglängden i nanometer på den laser som användes. Våglängden för de två lasrar som användes under laborationen bestämdes under en tidigare laboration. I den andra kolumnen visas $n$, alltså ordningstalet (n:te ordningens sidomaxima), som längden uppmättes utifrån. Den tredje kolumnen, $x$, anger i meter avståndet mellan de bägge n:te ordningens sidomaxima. Till sist visas $L$, längden mellan CD-skivan (vårt gitter) och ytan där laserns punkter syns.

In [1]:
# importera bibliotek som används i projektet
import numpy as np # fler mattefunktioner än Python har inbyggt
import matplotlib.pyplot as plt # plotta grafer!
import pandas as pd # läsa in CSV-filer och visa tabeller utifrån dem

In [2]:
# läs in laborationsdata
# datan läses in ifrån en CSV-fil
df = pd.read_csv("data.csv")
df # visa data som antecknades under labben

,Wavelength laser (nm),n,x (m),L (m)
0,663,1,0.282,0.287
1,663,1,0.625,0.648
2,631,1,0.604,0.648
3,631,1,0.302,0.323
4,631,2,1.030,0.323


Med hjälp av mätdatan kan gitterkonstanten $d$ beräknas. Gitterkonstanten kan beräknas med hjälp av varje rad i tabellen, vilket betyder att det kommer finnas lika många värden för gitterkonstanten som antalet mätvärden. Då samma CD-skiva används ska gitterkonstanten ha samma värde för samtliga mätvärden.

In [3]:
# funktion som beräknar spårbredden (gitterkonstanten) utifrån mätvärden i en given rad
def calculate_lattice_constant(row):
    # vinkeln i triangeln som uppstår mellan ytan som laserns
    # punkter syns på och avståndet mellan CD-skivan och ytan
    angle = np.arctan((row["x (m)"]/2)/row["L (m)"])

    # enligt d * sin(alpha) = n * lambda
    # så kan vi beräkna d, gitterkonstanten eller spårbredden, på detta sätt
    d = (row["n"] * row["Wavelength laser (nm)"]) / np.sin(angle)

    return d

# med hjälp av df["Latic constant, d (nm)"] skapas en ny kolumn i vår befintliga tabell df
# och med df.apply körs en given funktion för varje rad i tabellen
# värdena som syns i den nya kolumnen är alltså beräknade utifrån värdena i varje rad i tabellen
# axis=1 betyder att funktionen körs för varje rad och inte för varje kolumn (axis=0)
df["Lattice constant, d (nm)"] = df.apply(lambda row: calculate_lattice_constant(row), axis=1)

# skriv ut tabellen igen så vi kan se den nya kolumnen "Latic constant, d (nm)"
# med dess beräknade värden
df

,Wavelength laser (nm),n,x (m),L (m),"Lattice constant, d (nm)"
0,663,1,0.282,0.287,1503.578386
1,663,1,0.625,0.648,1526.314267
2,631,1,0.604,0.648,1493.752880
3,631,1,0.302,0.323,1489.966265
4,631,2,1.030,0.323,1489.673457


In [4]:
# beräkna och skriv ut medelvärdet av gitterkonstanten för samtliga försök
# med np.round() avrundas medelvärdet till närmsta heltal
# int() används för att skriva ut det som ett heltal och inte .0 i slutet
df_mean = int(np.round(df[f"Lattice constant, d (nm)"].mean()))
print(f"Medelvärde: {df_mean} nm")

Medelvärde: 1501 nm


Utifrån laborationens resultat är gitterkonstanten, alltså spårbredden, 1501 nm. Det betyder att det är 1501 nm mellan varje spår på CD-skivan enligt laborationens resultat.

In [5]:
# beräkna och skriv ut standardavvikelse
std = df[f"Lattice constant, d (nm)"].std()
print(f"Standardavvikelse: {std}")
print(f"Förhållande mellan standardavvikelsen och medelvärdet: {np.round(std/df_mean*100)}%")

Standardavvikelse: 15.406384610091079
Förhållande mellan standardavvikelsen och medelvärdet: 1.0%


Standardavvikelse är ett mått på hur mycket mätvärdena avviker från ett medelvärde. ${}^{1}$ Om förhållandet mellan standardavvikelsen och medelvärdet är lågt, betyder det att det att mätvärdena avviker lite från medelvärdet. Desto lägre förhållande mellan standardavvikelsen och medelvärdet, desto troligare är det att den faktiska gitterkonstanten ligger mycket nära det medelvärde som tagits fram i denna laboration. Förhållandet är i detta fall endast cirka $1,0\%$ vilket gör laborationens resultat trovärdigt.

### Felkällor

Laborationen har flertalet felkällor. För det första kan eventuella fel bero på mätutrustning. Linjalen som användes under laborationen för att mäta avståndet mellan laserns punkter och mellan CD-skivan och ytan var en enmeterslinjal. Denna linjal kan antas ha en noggrannhet på cirka 1 millimeter. Detta på grund av graderingen på linjalen. Denna felmarginal går det att räkna på. T.ex. kan felmarginalen läggas till på alla $x$ och subtraheras på alla $L$, se nedan.

In [6]:
# funktion som lägger på en felmarginal på en given kolumn
def add_error_margin(row, column, error_margin):
    return row[column] + error_margin

# vi skapar en ny DataFrame (tabell) utifrån den gamla
# tabellen så att vi kan göra justeringar till tabellens värden
# utan att påverka den initial tabellen
df2 = pd.DataFrame(df)

# lägg till 0,005 m på alla x
df2["x (m)"] = df2.apply(lambda row: add_error_margin(row, "x (m)", 0.001), axis=1)

# subtrahera 0,005 m på alla L
df2["L (m)"] = df2.apply(lambda row: add_error_margin(row, "L (m)", -0.001), axis=1)

# beräkna gitterkonstanten för alla försök
df2["Lattice constant, d (nm)"] = df2.apply(lambda row: calculate_lattice_constant(row), axis=1)

# beräkna och skriv ut medelvärdet av gitterkonstanten för samtliga försök
# med np.round() avrundas medelvärdet till närmsta heltal
# int() används för att skriva ut det som ett heltal och inte .0 i slutet
df2_mean = int(np.round(df2[f"Lattice constant, d (nm)"].mean()))
print(f"Nytt medelvärde med felmarginal: {df2_mean} nm")
print(f"Felmarginal på medelvärdet jämfört med faktiska resultat: +/- {df_mean - df2_mean} nm")
print(f"Felmarginalens förhållande till resultat: {np.round((df_mean - df2_mean) / df_mean * 100, decimals=3)}%")

df2

Nytt medelvärde med felmarginal: 1495 nm
Felmarginal på medelvärdet jämfört med faktiska resultat: +/- 6 nm
Felmarginalens förhållande till resultat: 0.4%


,Wavelength laser (nm),n,x (m),L (m),"Lattice constant, d (nm)"
0,663,1,0.283,0.286,1495.098834
1,663,1,0.626,0.647,1522.429337
2,631,1,0.605,0.647,1489.834866
3,631,1,0.303,0.322,1482.162248
4,631,2,1.031,0.322,1487.967294


Felmarginalen är alltså cirka $0,4\%$ av det faktiska resultatet vilket gör denna felkälla försumbar.

En annan felkälla kan vara det faktum att lasern som användes under laborationen inte var helt monokromatisk. Att en laser är monokromatisk betyder att den sänder ut en enda färg, alltså att allt ljus som kommer från lasern har exakt samma våglängd. En laser är inte helt monokromatisk, men nästan helt monokromatisk. ${}^{2}$ Våglängden på ljuset som sänds ut kan ha lite varierande våglängd och därför ha inverkan i resultatet.

- Felaktig våglängd på lasern. Laserns våglängd uppmättes under en annan laboration. Hur påverkar det resultatet?
- ~~Felaktigt uppmätta värden. Mänsklig faktor.~~
- ~~Låg noggrannhet på linjal.~~
- Ej horisontellt liggande CD-skiva.
- Ej rakt uppåt pekande laser.
- ~~Ej monokromatisk laser.~~
- Ojämnheter i CD-skivans yta (repig).

## Källhänvisningar

1. Nationalencyklopedin. *Standardavvikelse*. Hämtad 2021-03-31 från https://www.ne.se/uppslagsverk/encyklopedi/l%C3%A5ng/standardavvikelse
1. Laser Radiation Properties. (9 juni 2020). Hämtad 2021-03-31 från https://chem.libretexts.org/@go/page/74652